# Vaccination Report

Use Case: OSHA requires business leaders to have a general understanding of where their sites stand in terms of vaccination status.

This report helps pull together the necessary information to get this information.

Process Flow:
 1. Get the vaccine report from Benefits team member and load into the script directory. 
 
 a. Ensure the file name is in the format: 
          VN_Vaccination Report_20211020.xlsx
          Where the 20211020 is yyyymmdd
 2. Run the script.

In [2]:
import pyodbc
import pandas as pd, IPython.display, ipywidgets as widgets
import os
import re
import datetime as dt
from datetime import date
import panel as pn
import numpy as np
pd.set_option("display.max_columns", None) #display all columns when viewing dataframe

ModuleNotFoundError: No module named 'ipywidgets'

## Rename the vaccine files to standard format

In [3]:
for filename in os.listdir("."):
    if filename.startswith("VN_Vaccination"):
        os.rename(filename, filename[0:30] + ".csv")

## Feed in the necessary data

You require access to *SPCPRDRPTDB > EHS_Replication* db & *EDWPRDRPTDB > EDW db*

#### HR data is needed for headcount, employee status, and other employee information.

#### TRT data is needed to extract regions (for Energy and SSD)

In [3]:
# HR Nonsensitive data used for headcount purposes
conn_hr = pyodbc.connect('Driver={SQL Server};'
                      'Server=EDWPRDRPTDB;'
                      'Database=EDW;'
                      'Trusted_Connection=yes;')

cursor_hr = conn_hr.cursor()

hr = pd.read_sql_query("Select Employee_Non_Sensitive_Key, File_Number, Worker_Type, Is_Conversion, Employee_Name, Last_Name, First_Name, Middle_Name, Employee_Type, Contingent_Worker_Type, Time_Type, Is_Active, Status, Job_Profile, Business_Title, Department_Name, Primary_Work_Email, Display_Name, Country, Cost_Center, SIO_Code, Location, Location_ID, Management_Level, Is_Exempt, SIO_Code_Description, User_Name, Location_Postal_Code, Location_City, Location_State, Location_Country, Cost_Center_Hierarchy, Job_Family, Length_of_Service, Location_Type, Manager_Preferred_Name, Manager_ID, Manager_Full_Legal_Name, Recruiter, From_Which_Merger, Pay_Rate_Type, Source_Deleted, Last_Day_of_Work, Is_NonInternEmployee, Zone, Is_Latest_For_UserName, Labor_Type, Is_ValidHeadCount, Hire_Date, Original_Hire_Date FROM [EDW].[DimEmployeeNonSensitive]",conn_hr)
trt = pd.read_sql_query("Select TRT_ID, Name, Total_Sq_Ft, Office_Sq_Ft, System_Name, Type, Status, Created_Date, criticality, Site_Code, Phone, Address, City, State, Zip, Country, Address_Number, Address_Street, Latitude, Longitude, Lease_Start_Date, Lease_End_Date, Region, Supercharger_Region, Country_Description, Associated_TRT_ID, Sales_Metro, TRT_Key, Types, Functions, Attributes, Service_Close_Date, Service_City, Service_Metro, Service_POC, Service_POC_Email, ServiceManagerName, Service_Zone, Service_Territory FROM [EDW].[dimTRTstore]",conn_hr)

cursor_hr.close()
conn_hr.close()

#### COVID Status Data is needed to calculate COVID Positivity rate. 

* COVID Positivity (as agreed upon by Mary Jo Press & Laurie Shelby) is:

    Total COVID+ cases for all employees (including terminated) 
    / 
    Total Active Employees

In [4]:
# # Pandemic table
# conn_pt = pyodbc.connect('Driver={SQL Server};'
#                       'Server=SPCPRDRPTDB;'
#                       'Database=EHS_Replication;'
#                       'Trusted_Connection=yes;')

# cursor_pt = conn_pt.cursor()

# covid = pd.read_sql_query('SELECT * FROM [dbo].[Pandemic_data_with_HR_data_log]',conn_pt)
# cursor_pt.close()
# conn_pt.close()

In [5]:
# pyodbc connection string
conn = pyodbc.connect("DRIVER={Vertica};SERVER=utopia.teslamotors.com;PORT=5433;DATABASE=utopia;UID=yourusername;PWD=yourpassword;")

# Define Cursor
cus=conn.cursor()

# Execute SQL statement to get current datetime and store result in cursor
pandemic = pd.read_sql_query('SELECT * FROM EHSDM.VW_Dim_Pandemic',conn)
cus.close()
conn.close()


#### Vaccination Status

Vaccine data has to be fed in manually from spreadsheets sent by the benefits team. These are loaded into the notebook's file directory for ease of access.

The data is sent daily (some days are missed occasionally and weekends are not included) and it is a snapshot in time for the number of vaccinated persons in NA.

In [4]:
file_list = os.listdir()
available_files = []
print("Dates of filese available:")
for file in file_list:
    if "VN_" in file:
        available_files.append(file[-8:-6]+"-"+file[-6:-4]+"-"+file[-12:-8])
        print(file[-8:-6]+"-"+file[-6:-4]+"-"+file[-12:-8])
available_files = [dt.datetime.strptime(date, "%m-%d-%Y").date() for date in available_files]

Dates of filese available:
10-01-2021
10-04-2021
10-05-2021
10-07-2021
10-08-2021
10-11-2021
10-12-2021
10-13-2021
10-14-2021
10-15-2021
10-18-2021
10-19-2021
10-25-2021
10-26-2021
10-27-2021
10-28-2021
10-29-2021
11-01-2021
11-02-2021
11-03-2021
11-04-2021
11-05-2021
11-08-2021
11-09-2021
11-10-2021
11-11-2021
11-12-2021
11-15-2021
11-16-2021
11-17-2021
11-18-2021
11-19-2021
11-22-2021
11-24-2021
11-30-2021
12-01-2021
12-02-2021
12-03-2021
12-06-2021
12-07-2021
12-09-2021
12-10-2021
12-13-2021
12-14-2021
12-15-2021
12-16-2021
12-17-2021
12-20-2021
12-21-2021
12-22-2021
12-23-2021
12-28-2021
12-29-2021
12-30-2021
01-04-2022
01-05-2022
01-06-2022
01-07-2022
01-10-2022
01-11-2022
01-12-2022
01-19-2022
01-20-2022
01-21-2022
01-24-2022
01-26-2022
01-27-2022
01-28-2022
01-31-2022
02-01-2022
02-03-2022
02-04-2022
02-07-2022
02-08-2022
02-09-2022
02-10-2022
02-11-2022
02-14-2022
02-15-2022
02-16-2022
02-17-2022
02-18-2022
02-22-2022
02-23-2022
02-26-2022
02-27-2022
02-28-2022
03-01-2022
03-02

In [5]:
pn.extension()

initial_date = pn.widgets.DatePicker(name='Initial Date', enabled_dates=available_files)
initial_date

DatePicker(enabled_dates=[datetime.date(2021, ...], name='Initial Date')

In [7]:
date_compare = pn.widgets.DatePicker(name='Date of Comparison', enabled_dates=available_files)
date_compare

DatePicker(enabled_dates=[datetime.date(2021, ...], name='Date of Comparison')

In [8]:
initial_date_string = initial_date.value.strftime("%Y%m%d")
compare_date_string = date_compare.value.strftime("%Y%m%d")

initial_date_string

'20220223'

In [9]:
initial_date_string = initial_date.value.strftime("%Y%m%d")
compare_date_string = date_compare.value.strftime("%Y%m%d")
try:
    initial_file = pd.read_excel('VN_Vaccination_Report_' + initial_date_string + '.csv')
    print("Initial File read correctly!")
except:
    print('The selected initial date vaccine report is not available or in the correct format.')
    
try:
    compare_file = pd.read_excel('VN_Vaccination_Report_' + compare_date_string + '.csv')
    print("Comparison File read correctly!")
except:
    print('The selected comparison date vaccine report is not available or in the correct format.')

The selected initial date vaccine report is not available or in the correct format.
The selected comparison date vaccine report is not available or in the correct format.


## Merging the datasets

The purpose of merging the datasets is to ensure that locations/departments are standardized across the datasets and to link key characteristics like region, employee status, employee type, and cost center hierachy across all the different datasets (because they are not initial present).

* COVID table > Adding (HROS Location, HR Department, TRT Region, HR Cost Center Hierachy, HR emp_type)
 * COVID & HR (linking based on employee username)
 * COVID & TRT (linking based on HR Location and TRT System Name*)


* HR table > Adding (TRT Region)
 * HR & TRT (linking based on HR Location and TRT System Name)


* Vaccine tables > Adding (HR Employee Status (active/inactive), TRT Region, HR emp_type)
 * Vaccine & HR (linking based on Vacc_Employee Email and HR_Employee Email)
 * Vaccine & TRT (linking based on Vacc_Location and TRT System Name)
 
*For Region, we use Location rather than state because on 10/20/21 an analysis showed that out of 889 unique cities in the HR database, 98 were not found in the TRT table, all of which would not have a region associated with them

In [ ]:
# covid.head()

In [ ]:
hr.head()

In [ ]:
trt.head()

In [ ]:
initial_file.head()

In [ ]:
pandemic.head()

In [11]:
# covid_copy = covid
pandemic_copy = pandemic
hr_copy = hr
trt_copy = trt
vaccine_initial = initial_file
vaccine_compare = compare_file

#stripping trailing spaces from all the files
trt_copy = trt_copy.applymap(lambda x: x.strip() if isinstance(x, str) else x)
# covid_copy = covid_copy.applymap(lambda x: x.strip() if isinstance(x, str) else x)
hr_copy = hr_copy.applymap(lambda x: x.strip() if isinstance(x, str) else x)
vaccine_initial = vaccine_initial.applymap(lambda x: x.strip() if isinstance(x, str) else x)
vaccine_compare = vaccine_compare.applymap(lambda x: x.strip() if isinstance(x, str) else x)

#converting employee ID to string 
hr_copy['File_Number'] = hr_copy['File_Number'].astype(str)
vaccine_initial['EmployeeID'] = vaccine_initial['EmployeeID'].astype(str)
vaccine_compare['EmployeeID'] = vaccine_compare['EmployeeID'].astype(str)


In [12]:
# # Adding columns to COVID table

# covid_copy = covid_copy.merge(hr_copy[['Employee_Type', 'Location', 'Department_Name', 'Cost_Center_Hierarchy', 'User_Name']], how='left', left_on='exp_empID', right_on='User_Name')
# covid_copy = covid_copy.merge(trt_copy[['System_Name', 'Region']], how='left', left_on='Location', right_on='System_Name')

In [13]:
# Adding columns to HR table

hr_copy = hr_copy.merge(trt_copy[['System_Name','Region']],left_on='Location', right_on = "System_Name", how='left')
hr_copy = hr_copy.merge(vaccine_initial[['WorkEmail', 'ApprovalStatus']],left_on='Primary_Work_Email', right_on = "WorkEmail", how='left')
hr_copy = hr_copy.merge(vaccine_compare[['WorkEmail', 'ApprovalStatus']],left_on='Primary_Work_Email', right_on = "WorkEmail", how='left', suffixes=('', '_compare'))


In [14]:
#Adding columns to the Vaccine tables

vaccine_initial = vaccine_initial.merge(hr_copy[['Is_Active', 'Employee_Type', 'Primary_Work_Email']], how='left', left_on='WorkEmail', right_on='Primary_Work_Email')
vaccine_initial = vaccine_initial.merge(trt_copy[['System_Name', 'Region']], how='left', left_on='WorkLocation', right_on='System_Name')
vaccine_initial.rename(columns={"Department": "BusinessType", "CostCenterDescription": "Department"}, inplace=True)

In [15]:
hr_copy.head()

,Employee_Non_Sensitive_Key,File_Number,Worker_Type,Is_Conversion,Employee_Name,Last_Name,First_Name,Middle_Name,Employee_Type,Contingent_Worker_Type,Time_Type,Is_Active,Status,Job_Profile,Business_Title,Department_Name,Primary_Work_Email,Display_Name,Country,Cost_Center,SIO_Code,Location,Location_ID,Management_Level,Is_Exempt,SIO_Code_Description,User_Name,Location_Postal_Code,Location_City,Location_State,Location_Country,Cost_Center_Hierarchy,Job_Family,Length_of_Service,Location_Type,Manager_Preferred_Name,Manager_ID,Manager_Full_Legal_Name,Recruiter,From_Which_Merger,Pay_Rate_Type,Source_Deleted,Last_Day_of_Work,Is_NonInternEmployee,Zone,Is_Latest_For_UserName,Labor_Type,Is_ValidHeadCount,Hire_Date,Original_Hire_Date,System_Name,Region,WorkEmail,ApprovalStatus,WorkEmail_compare,ApprovalStatus_compare
0,1,000051,Employee,False,Trevor Campbell,Campbell,Trevor,,Regular,Regular,Full time,False,Terminated,Production Associate I,Production Associate,Nevada - Model 3/Y - Drive Unit,TCampbell@tesla.com,Trevor Campbell,United States of America,30630200,RNODU,NA-US-NV-Sparks-Electric Avenue,782,S1 (I),False,Reno Production Drive Unit Shared,tcampbell,89434,Sparks,Nevada,United States of America,Manufacturing,Manufacturing,"5 year(s), 8 month(s), 30 day(s)",Manufacturing,Carl Cavner,727888,Carl Cavner,None,None,Hourly,False,2019-05-16,True,North America,True,Direct,False,2013-08-19,2013-08-19,NA-US-NV-Sparks-Electric Avenue,NA-US-Central West,NaN,NaN,NaN,NaN
1,2,000052,Employee,False,Matt Lyons,Lyons,Matt,None,Regular,Regular,Full time,False,Terminated,Technical Support Engineer,Field Technical Specialist,Field Technical Support Operations,mlyons@tesla.com,Matt Lyons,United States of America,30150800,SERVI,NA-US-TX-Austin-Pond Springs,658,P2 ( - ),True,Auto Service,mlyons,78759,Austin,Texas,United States of America,Service,Operations & Business Support,"7 year(s), 4 month(s), 30 day(s)",None,Andrew Pilger,153872,Andrew Pilger,None,None,Salary,False,2021-03-05,True,North America,True,Overhead,False,2013-10-07,2013-10-07,NA-US-TX-Austin-Pond Springs,NA-US-Texas,NaN,NaN,NaN,NaN
2,3,000053,Employee,False,Preston Danna,Danna,Preston,None,Regular,Regular,Full time,False,Terminated,"Service Technician IV, Auto","Senior Technician, Mobile Service",North America Mobile Service,pdanna@tesla.com,Preston Danna,United States of America,30150200,D1000,NA-US-TX-Austin-Mobile Service,5483,S4 (IV),False,Stores & Service Centers,pdanna,78750,Austin,Texas,United States of America,Service,Vehicle Service,"6 year(s), 1 month(s), 22 day(s)",None,Anthony Campos,659041,Anthony Campos,None,None,Hourly,False,2019-12-13,True,North America,True,Indirect,False,2013-10-23,2013-10-23,NA-US-TX-Austin-Mobile Service,NA-US-Texas,NaN,NaN,NaN,NaN
3,4,000054,Employee,False,Tim Gonzales,Gonzales,Tim,None,Regular,Regular,Full time,False,Terminated,Service Support Associate II,Service Porter,North America Service- Inactive,tgonzales@teslamotors.com,Tim Gonzales,United States of America,30150000,C9675,NA-US-AZ-Scottsdale-Hayden Road,72,S2 (II),False,NA-US-AZ-Scottsdale-Service,000054,85260,Scottsdale,Arizona,United States of America,Service,Vehicle Service,"1 year(s), 4 month(s), 16 day(s)",None,Nicholas Thompson,153311,Nicholas Thompson,None,None,Hourly,False,2015-02-25,True,North America,True,Overhead,False,2013-10-09,2013-10-09,NA-US-AZ-Scottsdale-Hayden Road,NA-US-Central West,NaN,NaN,NaN,NaN
4,5,000055,Employee,False,Colby Reynolds,Reynolds,Colby,None,Regular,Regular,Full time,False,Terminated,Service Support Associate II,Service Assistant,North America Service Centers,chreynolds@tesla.com,Colby Reynolds,United States of America,30150100,D1000,NA-US-TX-Dallas-Cedar Springs Road,940,S2 (II),False,Stores & Service Centers,chreynolds,75235,Dallas,Texas,United States of America,Service,Vehicle Service,"3 year(s), 11 month(s), 29 day(s)",None,Jimy Wright,656094,Jimy Wright,None,None,Hourly,False,2017-10-11,True,North America,True,Overhead,False,2013-10-21,2013-10-21,NA-US-T

## Creating a universal data set

### Commonizing the table to enable visualization and ease of adaptation

| Employee Identifier | Characteristic Column(s)|
| ----------- | ----------- |
| All Active & Inactive Employees at Tesla     | Characteristic      |


The vaccination and potentially COVID rate is broken down to employees and the metrics are grouped by different characteristics (location, department, COVID status, employee status, contingent employee type, etc.).

There are many requirements (or filters) requested by the customers to change the summation of employees based on certain conditions. Thus this characterisitic table type enable conditional summation across various data visualization tools (such as Excel, Tableau).

In [16]:
# import numpy as np
# from datetime import date

# #COVID_Pos_afterApril is a column added to test if that person was tested positive for COVID
# #after April 1, 2021 when the vaccine became widely available at Tesla
# apriltest = pd.to_datetime('2021-06-01').normalize()
# covid_copy['afterApril_testidentifier'] = (apriltest - covid_copy['Date of Test']).dt.days
# covid_copy['COVID_Pos_afterApril'] = np.where(np.logical_and(covid_copy['exp_status']=='Positive', covid_copy['afterApril_testidentifier']<0), 1, 0)
# hr_copy = hr_copy.merge(covid_copy[['User_Name','COVID_Pos_afterApril']],left_on='User_Name', right_on = "User_Name", how='left')

# #Creating a days worked column to calculate how employee tenure in days
# today = pd.to_datetime('today').normalize()
# hr_copy['Original_Hire_Date'] = pd.to_datetime(hr_copy['Original_Hire_Date'])
# hr_copy['Days_Worked'] = (today - hr_copy['Original_Hire_Date']).dt.days

In [17]:
import numpy as np
from datetime import date

#Most recent positive test date column added to hr_copy
pandemic_copy_pos = pandemic_copy[pandemic_copy['TestHistory']=="Positive"]
pandemic_copy_pos = pandemic_copy_pos.sort_values(by='TestResultDate', ascending=False)
pandemic_copy_pos = pandemic_copy_pos.drop_duplicates(subset='EmployeeUserName', keep='first')
pandemic_copy_pos.TestResultDate.fillna(pandemic_copy_pos.QuarantineStartDate, inplace=True)
hr_copy_pos = hr_copy.merge(pandemic_copy_pos[['EmployeeUserName','TestResultDate']],left_on='User_Name', right_on = "EmployeeUserName", how='left')

In [18]:
# import numpy as np
# from datetime import date

# #Most recent positive test date column added to hr_copy
# covid_copy_pos = covid_copy[covid_copy['exp_status']=="Positive"]
# covid_copy_pos = covid_copy_pos.sort_values(by='Date of Test Results', ascending=False)
# covid_copy_pos = covid_copy_pos.drop_duplicates(subset='exp_empID', keep='first')
# hr_copy_pos = hr_copy.merge(covid_copy_pos[['exp_empID','Date of Test Results']],left_on='User_Name', right_on = "exp_empID", how='left')

In [19]:
#Creating a days worked column to calculate how employee tenure in days 
today = pd.to_datetime('today').normalize()
hr_copy_pos['Original_Hire_Date'] = pd.to_datetime(hr_copy_pos['Original_Hire_Date'])
hr_copy_pos['Days_Worked'] = (today - hr_copy_pos['Original_Hire_Date']).dt.days

In [20]:
#Only keeping US employee records
hr_copy_pos = hr_copy_pos[hr_copy_pos['Country'] == 'United States of America']


In [21]:
hr_copy_pos.to_csv("test_pos_04.15.csv")


## Calculating the Metrics & Generating the Tables



### Calculating headcount for locations, departments, and regions

Need headcount for:
* All Tesla locations
* All location departments
* NA Regions (for Energy and SSD)


In [ ]:
#count for all business types in US for active employees
business_types = vaccine_initial['BusinessType'].unique()
bsn_hc = {}
for bsn in business_types:
    b = hr_copy.query('Cost_Center_Hierarchy == "' + bsn + '" & Is_Active == True & Location_Country == "United States of America"')['Primary_Work_Email'].count()
    bsn_hc[bsn] = b

business_hc = pd.DataFrame.from_dict(bsn_hc, orient='index', columns=['Headcount'])
business_hc

In [ ]:
#headcount for all locations
locations = vaccine_initial['WorkLocation'].unique()
loc_hc = {}
for loc in locations:
    l = hr_copy.query('Location == "' + loc + '" & Is_Active == True & Location_Country == "United States of America"')['Primary_Work_Email'].count()
    loc_hc[loc] = l
    
location_hc = pd.DataFrame.from_dict(loc_hc, orient='index', columns=['Headcount'])
location_hc = location_hc[(location_hc != 0).all(1)]
location_hc = location_hc.sort_values(['Headcount'], ascending=[False])
location_hc

In [ ]:
dep_hr = vaccine_initial
dep_hr['Loc_Dep'] = vaccine_initial['WorkLocation'] + '%' + vaccine_initial['Department']
comb_list = dep_hr['Loc_Dep']
comb_list = comb_list.unique()
comb_dept = pd.DataFrame(comb_list,columns =['Loc_Dep'])
comb_dept['Location'] = comb_dept['Loc_Dep'].str.split('%', expand=True)[0]
comb_dept['Department'] = comb_dept['Loc_Dep'].str.split('%', expand=True)[1]

In [ ]:
dep_hc = {}
for i in range(len(comb_dept["Location"])):
    d = hr_copy.query('Location == "' + comb_dept["Location"][i]+ '" & Department_Name == "' + comb_dept["Department"][i]+ '" & Is_Active == True & Location_Country == "United States of America"')['Primary_Work_Email'].count()
    l_d = comb_dept["Location"][i] + '%' + comb_dept["Department"][i]
    dep_hc[l_d] = d

dep_hc


In [ ]:
department_hc = pd.DataFrame.from_dict(dep_hc, orient='index', columns=['Headcount'])
department_hc = department_hc.reset_index()

comb_dept1 = comb_dept.merge(department_hc, how='left', left_on='Loc_Dep', right_on='index')
comb_dept1.drop(columns=['Loc_Dep', 'index'])

In [ ]:
comb_dept2 = comb_dept1.merge(trt_copy[['System_Name','Region']], how='left', left_on='Location', right_on='System_Name')
comb_dept2

In [ ]:
comb_dept[comb_dept['Location'] == 'NA-US-CA-Fremont-45500 Fremont Blvd']['Headcount'].sum()

In [ ]:
vaccine_initial.head()